我们先定义一个与上一节中相同的含单个隐藏层的多层感知机

In [6]:

from mxnet import init,nd
from mxnet.gluon import nn

model = nn.Sequential()

model.add(nn.Dense(256,activation='relu'))
model.add(nn.Dense(10))

model.initialize() ##默认方式初始化
X =nd.random.uniform(shape=(2,20))
y = model(X)





[[ 0.0215123  -0.00999171  0.01383718 -0.0259273  -0.00332951  0.02987511
  -0.01507707 -0.00939399 -0.09489147 -0.07161631]
 [ 0.04690181  0.00833477 -0.03466241  0.03673035 -0.03812394  0.02809485
   0.01214801  0.03816549 -0.08725169 -0.01590922]]
<NDArray 2x10 @cpu(0)>


In [10]:
#访问模型参数
model[0].params,type(model[0].params)
##访问模型的具体参数
model[0].params['dense6_weight']

Parameter dense6_weight (shape=(256, 20), dtype=float32)

In [12]:
##查看参数的具体值
model[0].weight.data()
#查看参数的梯度
model[0].weight.grad()


[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
<NDArray 256x20 @cpu(0)>

In [15]:
#使用collect_params函数来获取net变量所有嵌套
model.collect_params()
model.collect_params('.*weight') ##使用正则表达式来获取想要的参数

sequential3_ (
  Parameter dense6_weight (shape=(256, 20), dtype=float32)
  Parameter dense7_weight (shape=(10, 256), dtype=float32)
)

In [16]:
model.initialize(init=init.Normal(sigma=0.01),force_reinit=True)
model[0].weight.data()[0]


[ -6.54585892e-03   5.46171237e-03  -3.14823980e-03   6.53286325e-03
   2.39426945e-03   1.43013904e-02   8.93270038e-03   1.07622147e-02
  -4.81205340e-03   4.11936874e-03   5.70393493e-03  -3.80902132e-03
  -5.08007919e-03  -1.39692018e-03  -9.47653371e-06   8.14091600e-03
  -1.76932309e-02  -1.07601862e-02   8.86166934e-03  -1.24831125e-02]
<NDArray 20 @cpu(0)>

In [17]:
##常数初始化
model.initialize(init=init.Constant(1),force_reinit=True)
model[0].weight.data()[0]


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
<NDArray 20 @cpu(0)>

In [20]:
class MyInit(init.Initializer):
    def _init_weight(self,name,data):
        print('Init',name,data.shape)
        data[:] = nd.random.uniform(low=-10,high=10,shape=data.shape)
        data *= data.abs() >=5
model.initialize(MyInit(),force_reinit=True)
model[0].weight.data()[0]

Init dense6_weight (256, 20)
Init dense7_weight (10, 256)



[-0.         -8.59105587 -5.17162752  0.          0.         -0.
 -5.07873631  0.          0.          8.5375824   0.          7.07880211
 -0.         -0.          0.         -0.         -0.          6.45118904
  0.          0.        ]
<NDArray 20 @cpu(0)>

In [24]:
model = nn.Sequential()
shared = nn.Dense(8,activation='relu')
model.add(nn.Dense(8,activation='relu'),
          shared,
          nn.Dense(8,activation='relu',params=shared.params),
          nn.Dense(10))
model.initialize(init=init.Normal(sigma=0.01))
x = nd.random.uniform(shape=(2,20))
model(x)
model[1].weight.data()[0] == model[2].weight.data()[0]


[ 1.  1.  1.  1.  1.  1.  1.  1.]
<NDArray 8 @cpu(0)>